# Clusterização
***

Os algoritmos de clusterização ou agrupamento permitem que seja feito agrupamento de grupos com base nas semelhanças encontradas. É uma técnica que permite realizar a divisão de grupos em um conjunto de dados de forma automática baseado em medidas de similaridade ou de distância. Existem vários métodos que permitem obter medidas de similaridade, podemos citar a similaridade de cosseno e a correlação de Pearson.

É um dos algoritmos mais utilizados para realizar agrupamentos de dados numéricos em mineração de dados

#### São utilizados para:

- Segmentação de mercado
- Agrupamento de características de clientes, produtos mais similares
- Agrupar perfis de clientes (Amazon Prime, Netflix, similares)
- Recomendação de itens


### Definição
- A clusterização, que é uma técnica de aprendizado não supervisionado que permite dividir automaticamente o conjunto de dados em grupos de acordo com uma similaridade.


Exemplo prático:
![](doc_img/cluster.png)

## Implementando algoritmo de K-means

Nesta atividade, vamos implemenar o algoritmo K-means em um conjunto de dados. Para isso, vamos executar os seguintes passos:


- Carregar o dataset de iris;
- Análisar dos dados coletados;
- Realizar o tratamento de dados;
- Aplicar modelo ML de K-means;
- Visualizar os resultados obtidos;

## Importantando as bibliotecas

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px  #Criação de graficos dinâmnicos
import plotly.graph_objects as go #Para criação e concatenização de graficos
from sklearn.preprocessing import StandardScaler # para relizar a padronização dos dados
from sklearn.cluster import KMeans
import sklearn
import plotly

In [2]:
print(f'''Verificando versão das bibliotecas utilizadas:\n
pandas: {pd.__version__ }
numpy: {np.__version__ }
sklearn: {sklearn.__version__ }
plotly: {plotly.__version__}
''')


Verificando versão das bibliotecas utilizadas:

pandas: 2.2.2
numpy: 2.0.2
sklearn: 1.6.1
plotly: 5.24.1



### Coletando dados

In [6]:
PATH = R'/content/iris.csv'
iris = pd.read_csv(PATH, sep = ';')

### Analisando os dados coletados

In [8]:
# Visualizando arquivos da base de dados
iris.tail()

,sepal length,sepal width,petal length,petal width,Class
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica


<img src=doc_img/flowers.png style='width:400px;height:250px'>

In [9]:
# Gerando algumas estatisticas da base de dados
iris.describe()

,sepal length,sepal width,petal length,petal width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [10]:
# Número de classes existentes no dataframe
iris['Class'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [11]:
# Contanto numero de classes
len(iris['Class'].unique())

3

In [ ]:
# Avaliando número de linhas e colunas do dataset
iris.shape

(150, 5)

In [12]:
# Verificando se existe algum valor nulo ou ausente
iris.isna().sum()

,0
sepal length,0
sepal width,0
petal length,0
petal width,0
Class,0


In [13]:
# Verificando informações da base de dados
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal length  150 non-null    float64
 1   sepal width   150 non-null    float64
 2   petal length  150 non-null    float64
 3   petal width   150 non-null    float64
 4   Class         150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


# Criando clusters com os dados das pétalas
***

Para o nosso primeiro experimento vamos realizar o agrupamento dos dados baseado no comprimento e na largura das pétalas.

### Coletando dados das pétalas

In [14]:
# criando variável X com as colunas petal length	petal width
X_petala = iris.iloc[:,[2,3]].values
X_petala[:10]

array([[1.4, 0.2],
       [1.4, 0.2],
       [1.3, 0.2],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.7, 0.4],
       [1.4, 0.3],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.5, 0.1]])

## Normalizando os dados
Quandos os dados estão em uma escala muito diferente, precisamos normalizar os dados colocando no mesmo padrão e na mesma escala.


Como o Kmeans realiza os cálculos baseado na distância, devemos padronizar os dados afim de que um atributo não seja considerado mais importanet do que o outro.

In [15]:
normalizar_dados = StandardScaler()
X_petala = normalizar_dados.fit_transform(X_petala)
X_petala[:10]

array([[-1.3412724 , -1.31297673],
       [-1.3412724 , -1.31297673],
       [-1.39813811, -1.31297673],
       [-1.2844067 , -1.31297673],
       [-1.3412724 , -1.31297673],
       [-1.17067529, -1.05003079],
       [-1.3412724 , -1.18150376],
       [-1.2844067 , -1.31297673],
       [-1.3412724 , -1.31297673],
       [-1.2844067 , -1.4444497 ]])

## Calculando o número de cluster
Para calcular o número de cluster vamos utilizar o método do cotovelo

WCSS é a soma da distância quadrada entre cada ponto e o centróide em um cluster

In [17]:
wcss_petala = [] # Cria uma lista vazia
for i in range(1,11):
  kmeans_petala = KMeans(n_clusters=i, random_state=0) # Executa o kmeans para todos os clusters e random_state = 0  para fixar e obter os mesmos resultados
  kmeans_petala.fit(X_petala) # realiza o treinamento
  wcss_petala.append(kmeans_petala.inertia_) # adiciona na lista os valores de wcss


In [18]:
#Visualizando os valores de wcss
for i in range(len(wcss_petala)):
  print('Cluster:', i ,'- Valor do wcss:', wcss_petala[i] )


Cluster: 0 - Valor do wcss: 300.0000000000001
Cluster: 1 - Valor do wcss: 54.14584701344988
Cluster: 2 - Valor do wcss: 18.046983891906265
Cluster: 3 - Valor do wcss: 12.307440251261838
Cluster: 4 - Valor do wcss: 9.192414757507374
Cluster: 5 - Valor do wcss: 7.227887553963048
Cluster: 6 - Valor do wcss: 6.699766804805106
Cluster: 7 - Valor do wcss: 6.200345405305922
Cluster: 8 - Valor do wcss: 4.93665777737511
Cluster: 9 - Valor do wcss: 4.533503232466192


## Definindo melhor número de clusters

In [19]:
grafico_cotovelo_petala = px.line(x= range(1,11), y=wcss_petala)
grafico_cotovelo_petala.update_xaxes(title_text="Número de Clusters")
grafico_cotovelo_petala.update_yaxes(title_text="Valor WCSS")
grafico_cotovelo_petala.update_layout(title_text="Definindo Número de Clusters", title_x=0.5)
grafico_cotovelo_petala.show()

## Executando o algoritmo K-means para as pétalas

In [20]:
kmeans_petala = KMeans(n_clusters=3, random_state=0)
label_cluster_petala = kmeans_petala.fit_predict(X_petala)

In [21]:
#Verifica a classificação dos clusters
label_cluster_petala

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

## Visualizando o centroides de cada cluster

In [22]:
centroides_petala = kmeans_petala.cluster_centers_
centroides_petala

array([[ 0.30564587,  0.16609419],
       [-1.30487835, -1.25512862],
       [ 1.02813193,  1.12749028]])

## Visualizando dados agrupados com dados das pétalas

In [25]:
grafico_petala = px.scatter(x = X_petala[:,0], y=X_petala[:,1], color= label_cluster_petala)
grafico_centroide_petala = px.scatter(x = centroides_petala[:,0], y=centroides_petala[:,1], size=[7,7,7])
grafico_final_petala = go.Figure(data = grafico_petala.data + grafico_centroide_petala.data)
grafico_final_petala.update_xaxes(title_text="Comprimento")
grafico_final_petala.update_yaxes(title_text="Largura")
grafico_final_petala.update_layout(title_text="Visualizando cluteres criados", title_x=0.5)

grafico_final_petala.show()


# Criando clusters com os dados das sépalas
***

Agora vamos realizar o agrupamento dos dados baseado no comprimento e na largura das sépalas.

### Coletando dados das sépalas

In [28]:
# Realizando o agrupamento sepala
X_sepala = iris.iloc[:,[0,1]].values
X_sepala = normalizar_dados.fit_transform(X_sepala)
X_sepala[:5]

array([[-0.90068117,  1.03205722],
       [-1.14301691, -0.1249576 ],
       [-1.38535265,  0.33784833],
       [-1.50652052,  0.10644536],
       [-1.02184904,  1.26346019]])

## Calculando valor do WCSS

In [29]:
wcss_sepala = [] # Cria uma lista vazia
for i in range(1,11):
  kmeans_sepala = KMeans(n_clusters=i, random_state=0) # Executa o kmeans para todos os clusters e random_state = 0  para fixar e obter os mesmos resultados
  kmeans_sepala.fit(X_sepala) # realiza o treinamento
  wcss_sepala.append(kmeans_sepala.inertia_) # adiciona na lista os valores de wcss

In [30]:
#Visualizando os valores de wcss
for i in range(len(wcss_sepala)):
  print('Cluster:', i ,'- Valor do wcss:', wcss_sepala[i] )


Cluster: 0 - Valor do wcss: 300.00000000000006
Cluster: 1 - Valor do wcss: 214.80004445463572
Cluster: 2 - Valor do wcss: 103.80692090161443
Cluster: 3 - Valor do wcss: 79.83879949066282
Cluster: 4 - Valor do wcss: 63.370908146687114
Cluster: 5 - Valor do wcss: 55.99062559037006
Cluster: 6 - Valor do wcss: 48.447754565842544
Cluster: 7 - Valor do wcss: 41.11866957543779
Cluster: 8 - Valor do wcss: 38.241369474610046
Cluster: 9 - Valor do wcss: 30.117936364651335


In [31]:
grafico_cotovelo = px.line( x= range(1,11), y=wcss_sepala)
grafico_cotovelo.update_xaxes(title_text="Número de Clusters")
grafico_cotovelo.update_yaxes(title_text="Valor WCSS")
grafico_cotovelo.update_layout(title_text="Definindo Número de Clusters", title_x=0.5)

grafico_cotovelo.show()

## Executando o algoritmo K-means para as pétalas

In [32]:
kmeans_sepala = KMeans(n_clusters=3, random_state=0)
label_cluster_sepala = kmeans_sepala.fit_predict(X_sepala)

In [33]:
#Verifica a classificação dos clusters
label_cluster_sepala

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2,
       2, 2, 2, 2, 2, 2, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0], dtype=int32)

In [34]:
centroides_sepala = kmeans_sepala.cluster_centers_
centroides_sepala

array([[-0.11190209, -0.95528589],
       [ 1.09616533,  0.09718925],
       [-1.00206653,  0.89510445]])

## Visualizando dados agrupados com dados das sépalas

In [38]:
grafico_final_sepala = px.scatter(x = X_sepala[:,0], y = X_sepala[:,1], color= label_cluster_sepala)
grafico_centroide = px.scatter(x = centroides_sepala[:,0], y = centroides_sepala[:,1], size = [7,7,7])
grafico_final_sepala = go.Figure(data = grafico.data + grafico_centroide.data)
grafico_final_sepala.update_xaxes(title_text="Comprimento")
grafico_final_sepala.update_yaxes(title_text="Largura")
grafico_final_sepala.update_layout(title_text="Clusters", title_x=0.5)

grafico_final_sepala.show()


In [ ]:
centroides_sepala[:,1],

(array([ 0.48794755, -0.9598232 ,  0.10644536,  1.97546931]),)

# Criando função para visualização de dados

In [52]:
def gera_grafico(X, y, color, X_centroide, y_centroide):
    grafico = px.scatter(x=X, y=y, color=color)
    grafico_centroide = px.scatter(x = X_centroide, y = y_centroide, size = [7,7,7])
    grafico_final = go.Figure(data = grafico.data + grafico_centroide.data)
    grafico_final.update_xaxes(title_text="Comprimento")
    grafico_final.update_yaxes(title_text="Largura")
    grafico_final.update_layout(title_text="Clusters", title_x=0.5)
    grafico_final.show()

### Executando a função para as pétalas

In [53]:
X = X_petala[:,0]
y = X_petala[:,1]
color = label_cluster_petala
X_centroide = centroides_petala[:,0]
y_centroide = centroides_petala[:,1]

gera_grafico(X, y, color, X_centroide, y_centroide)

### Executando a função para as sépalas

In [54]:
X = X_sepala[:,0]
y = X_sepala[:,1]
color = label_cluster_sepala
X_centroide = centroides_sepala[:,0]
y_centroide = centroides_sepala[:,1]

gera_grafico(X, y, color, X_centroide, y_centroide)